
# Meta-Agent Compiler Demo — Auto-Generating Simple Agents

This notebook explores the idea of a **Meta-Agent** that:

> Takes a natural language specification  
> → Designs a tiny LangGraph agent  
> → Executes it on a query  

We keep it intentionally constrained but conceptually powerful.



## 1. Setup


In [ ]:

%pip install -q langgraph langchain-openai langchain

import os, textwrap
os.environ.setdefault("OPENAI_API_KEY", "sk-REPLACE_ME")



## 2. Meta-Spec: Describe the Agent

We define a simple **schema** for what we want:

- Task type (summarization, classification, rewrite, etc.)
- Whether it needs tools (here: none, just pure LLM)
- Routing pattern (single node for now)


In [ ]:

from dataclasses import dataclass

@dataclass
class AgentSpec:
    name: str
    mode: str   # e.g. "summarizer", "classifier"



## 3. A Simple Meta-Agent: Parse Spec → Build Graph

We define a **compiler function** that:

- Reads `AgentSpec`
- Constructs a LangGraph with one node wired to an LLM
- Returns the compiled graph


In [ ]:

from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_openai import ChatOpenAI

base_llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

def build_agent_from_spec(spec: AgentSpec):
    def node(state: MessagesState):
        user_q = state["messages"][0]["content"]
        if spec.mode == "summarizer":
            prompt = f"Summarize the following text in 3–5 bullet points:\n\n{user_q}"
        elif spec.mode == "classifier":
            prompt = (
                "Classify the following user request into one of: 'research', 'coding', 'planning', 'other'. "
                "Then explain your reasoning.\n\n"
                f"{user_q}"
            )
        else:
            prompt = f"Answer the following as a helpful assistant:\n\n{user_q}"

        resp = base_llm.invoke([{"role": "user", "content": prompt}])
        return {"messages": state["messages"] + [{"role": "assistant", "name": spec.name, "content": resp.content}]}

    builder = StateGraph(MessagesState)
    builder.add_node("core", node)
    builder.add_edge(START, "core")
    builder.add_edge("core", END)
    return builder.compile()



## 4. Generate and Run Two Different Agents

We generate:

- A **SummarizerAgent**  
- A **ClassifierAgent**  


In [ ]:

spec_summary = AgentSpec(name="SummarizerAgent", mode="summarizer")
spec_classifier = AgentSpec(name="ClassifierAgent", mode="classifier")

summary_agent = build_agent_from_spec(spec_summary)
classifier_agent = build_agent_from_spec(spec_classifier)

text = (
    "I am working on an agentic AI system that uses MCP tools, LangGraph orchestration, "
    "and RAG to answer questions about internal documents. I want to present this in a "
    "clear, executive-friendly way."
)

print("=== SummarizerAgent ===")
res1 = summary_agent.invoke({"messages": [{"role": "user", "content": text}]})
print(res1["messages"][-1]["content"])

print("\n=== ClassifierAgent ===")
res2 = classifier_agent.invoke({"messages": [{"role": "user", "content": text}]})
print(res2["messages"][-1]["content"])



## 5. Extending Toward a True Meta-Agent

To grow this into a real **Meta-Agent-Compiler**, you would:

- Allow spec to include:
  - Multiple nodes (planner, worker, critic)
  - Tool requirements (MCP endpoints, RAG stores)
  - Memory configuration
- Have the meta-agent:
  - Propose a graph structure
  - Validate it
  - Emit actual Python code and tests

This simple version is your **first step**: agents as *compiled artifacts* from natural language intent.
